<a href="https://colab.research.google.com/github/AllysonAndrey/Microdados_Enem/blob/main/Data_Normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sn
import matplotlib.pyplot as mpl
import seaborn as sb
#import pyodbc as bd

# Nova seção